#Подключение библиотек

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D, Input
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import random

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

#Далее надо будет заменить данные для обученния

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/4/train_merge.csv') 
test = pd.read_csv('/content/drive/MyDrive/4/testMy.csv', encoding = 'latin1')

In [ ]:
train.head()

,label,review
0,others,Don't worry I'm girl hmm how do I know if you...
1,angry,When did I? saw many times i think -_- No. I n...
2,others,By by Google Chrome Where you live
3,angry,U r ridiculous I might be ridiculous but I am ...
4,others,Just for time pass wt do u do 4 a living then ...


In [ ]:
train.shape

(30160, 2)

In [ ]:
label = train['label'].unique()
label

array(['others', 'angry', 'sad', 'happy'], dtype=object)

In [ ]:
train['review'].apply(lambda x:len(str(x).split())).max()

157

In [ ]:
label = train['label'].unique()
label

array(['others', 'angry', 'sad', 'happy'], dtype=object)

In [ ]:
y=train['label'].values
y = pd.get_dummies(y)
print('Shape of label tensor:', y)

Shape of label tensor:        angry  happy  others  sad
0          0      0       1    0
1          1      0       0    0
2          0      0       1    0
3          1      0       0    0
4          0      0       1    0
...      ...    ...     ...  ...
30155      0      0       1    0
30156      0      0       0    1
30157      0      0       1    0
30158      0      0       1    0
30159      0      0       1    0

[30160 rows x 4 columns]


In [ ]:
tmp = train['review'] + test['review']
tmp = tmp.astype(str)
tokenizer = text.Tokenizer(num_words=400000,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=" ")
max_len = 70
tokenizer.fit_on_texts(tmp)
word_index = tokenizer.word_index
len(word_index)

24886

In [ ]:
X = train['review'].values
X = tokenizer.texts_to_sequences(X)
X = sequence.pad_sequences(X, maxlen=max_len)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (30160, 70)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                  random_state=46, 
                                                  test_size=0.3, shuffle=True)

In [ ]:
print('x_train.shape: ' + str(x_train.shape),' y_train.shape: '+str(y_train.shape))
print('x_test.shape: ' + str(x_test.shape),' y_train.shape: '+str(y_test.shape))

x_train.shape: (21112, 70)  y_train.shape: (21112, 4)
x_test.shape: (9048, 70)  y_train.shape: (9048, 4)


#Embending

In [ ]:
embeddings_index = {}
f = open('/content/drive/MyDrive/4/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196018it [04:24, 8289.62it/s]

Found 2196017 word vectors.


In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 24886/24886 [00:00<00:00, 435525.92it/s]


#Simple RNN 

In [ ]:
model=tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=x_train.shape[1]),
  tf.keras.layers.Embedding(len(tokenizer.word_index)+1,32),
  tf.keras.layers.SimpleRNN(100),
  tf.keras.layers.Dense(4, activation='softmax')])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy','AUC','Precision','Recall'])    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 32)            796384    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               13300     
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 810,088
Trainable params: 810,088
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelResults = model.fit(x_train, y_train, epochs=5, batch_size=64,validation_split=0.1)

Epoch 1/5
297/297 [==============================] - 14s 41ms/step - loss: 0.9856 - accuracy: 0.6070 - categorical_accuracy: 0.6070 - auc: 0.8338 - precision: 0.7445 - recall: 0.4103 - val_loss: 0.6078 - val_accuracy: 0.7727 - val_categorical_accuracy: 0.7727 - val_auc: 0.9379 - val_precision: 0.8181 - val_recall: 0.7240
Epoch 2/5
297/297 [==============================] - 11s 38ms/step - loss: 0.4576 - accuracy: 0.8433 - categorical_accuracy: 0.8433 - auc: 0.9632 - precision: 0.8707 - recall: 0.8079 - val_loss: 0.5139 - val_accuracy: 0.8168 - val_categorical_accuracy: 0.8168 - val_auc: 0.9536 - val_precision: 0.8390 - val_recall: 0.7898
Epoch 3/5
297/297 [==============================] - 12s 39ms/step - loss: 0.2983 - accuracy: 0.9025 - categorical_accuracy: 0.9025 - auc: 0.9828 - precision: 0.9143 - recall: 0.8896 - val_loss: 0.5616 - val_accuracy: 0.8040 - val_categorical_accuracy: 0.8040 - val_auc: 0.9494 - val_precision: 0.8146 - val_recall: 0.7907
Epoch 4/5
297/297 [============

In [ ]:
SimpleRNNModel = Sequential()

SimpleRNNModel.add(Input(shape=x_train.shape[1]))
SimpleRNNModel.add(Embedding(len(tokenizer.word_index)+1,32))
SimpleRNNModel.add(SimpleRNN(100))
SimpleRNNModel.add(Dense(4, activation='softmax'))
#SimpleRNNModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
SimpleRNNModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy','AUC','Precision','Recall', tf.keras.metrics.PrecisionAtRecall(recall=0.8)])    
SimpleRNNModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 32)            796384    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               13300     
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 810,088
Trainable params: 810,088
Non-trainable params: 0
_________________________________________________________________


In [ ]:
SimpleRNNModelResults = SimpleRNNModel.fit(x_train, y_train, epochs=5, batch_size=64,validation_split=0.1)

Epoch 1/5
297/297 [==============================] - 38s 44ms/step - loss: 1.1407 - accuracy: 0.5391 - categorical_accuracy: 0.5391 - auc: 0.7528 - precision: 0.5994 - recall: 0.3398 - precision_at_recall: 0.3160 - val_loss: 0.6581 - val_accuracy: 0.7604 - val_categorical_accuracy: 0.7604 - val_auc: 0.9261 - val_precision: 0.8022 - val_recall: 0.7064 - val_precision_at_recall: 0.4820
Epoch 2/5
297/297 [==============================] - 12s 40ms/step - loss: 0.4783 - accuracy: 0.8349 - categorical_accuracy: 0.8349 - auc: 0.9604 - precision: 0.8701 - recall: 0.7957 - precision_at_recall: 0.5456 - val_loss: 0.5061 - val_accuracy: 0.8210 - val_categorical_accuracy: 0.8210 - val_auc: 0.9539 - val_precision: 0.8456 - val_recall: 0.7936 - val_precision_at_recall: 0.6751
Epoch 3/5
297/297 [==============================] - 12s 40ms/step - loss: 0.3055 - accuracy: 0.8993 - categorical_accuracy: 0.8993 - auc: 0.9828 - precision: 0.9143 - recall: 0.8823 - precision_at_recall: 0.7117 - val_loss: 0

# LSTM (Long Short Term Memory) Networks

In [ ]:
SimpleLSTMModel = Sequential()
SimpleLSTMModel.add(Input(shape=x_train.shape[1]))
SimpleLSTMModel.add(Embedding(len(tokenizer.word_index)+1,32))
SimpleLSTMModel.add(LSTM(100))
SimpleLSTMModel.add(Dense(4, activation='softmax'))
SimpleLSTMModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision','Recall', tf.keras.metrics.PrecisionAtRecall(recall=0.8)])
    
SimpleLSTMModel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 70, 32)            796384    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 849,988
Trainable params: 849,988
Non-trainable params: 0
_________________________________________________________________


In [ ]:
SimpleLSTMModelResults = SimpleLSTMModel.fit(x_train, y_train, epochs=5, batch_size=64,validation_split=0.1)

Epoch 1/5
297/297 [==============================] - 52s 157ms/step - loss: 1.1218 - accuracy: 0.5496 - precision: 0.6688 - recall: 0.2907 - precision_at_recall_1: 0.3179 - val_loss: 0.5548 - val_accuracy: 0.8007 - val_precision: 0.8400 - val_recall: 0.7557 - val_precision_at_recall_1: 0.5125
Epoch 2/5
297/297 [==============================] - 45s 153ms/step - loss: 0.4373 - accuracy: 0.8519 - precision: 0.8763 - recall: 0.8203 - precision_at_recall_1: 0.5830 - val_loss: 0.4706 - val_accuracy: 0.8324 - val_precision: 0.8447 - val_recall: 0.8139 - val_precision_at_recall_1: 0.7099
Epoch 3/5
297/297 [==============================] - 45s 153ms/step - loss: 0.3192 - accuracy: 0.8926 - precision: 0.9061 - recall: 0.8745 - precision_at_recall_1: 0.7418 - val_loss: 0.4696 - val_accuracy: 0.8338 - val_precision: 0.8541 - val_recall: 0.8120 - val_precision_at_recall_1: 0.7998
Epoch 4/5
297/297 [==============================] - 46s 153ms/step - loss: 0.2696 - accuracy: 0.9113 - precision: 0.9

#GRU Gated Recurrent Units

In [ ]:
SimpleGRUModel = Sequential()
SimpleGRUModel.add(Input(shape=x_train.shape[1]))
SimpleGRUModel.add(Embedding(len(tokenizer.word_index)+1,32))
SimpleGRUModel.add(GRU(100))
SimpleGRUModel.add(Dense(4, activation='softmax'))
SimpleGRUModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision','Recall', tf.keras.metrics.PrecisionAtRecall(recall=0.8)])
    
SimpleGRUModel.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 70, 32)            796384    
_________________________________________________________________
gru (GRU)                    (None, 100)               39900     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 836,688
Trainable params: 836,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
SimpleGRUModelResults = SimpleGRUModel.fit(x_train, y_train, epochs=5, batch_size=64,validation_split=0.1)

Epoch 1/5
297/297 [==============================] - 45s 132ms/step - loss: 1.0584 - accuracy: 0.5750 - precision: 0.6696 - recall: 0.3378 - precision_at_recall_2: 0.3365 - val_loss: 0.5028 - val_accuracy: 0.8258 - val_precision: 0.8499 - val_recall: 0.7936 - val_precision_at_recall_2: 0.5867
Epoch 2/5
297/297 [==============================] - 38s 129ms/step - loss: 0.4073 - accuracy: 0.8625 - precision: 0.8812 - recall: 0.8394 - precision_at_recall_2: 0.6579 - val_loss: 0.4691 - val_accuracy: 0.8329 - val_precision: 0.8579 - val_recall: 0.8087 - val_precision_at_recall_2: 0.7595
Epoch 3/5
297/297 [==============================] - 38s 127ms/step - loss: 0.3091 - accuracy: 0.8944 - precision: 0.9083 - recall: 0.8828 - precision_at_recall_2: 0.7863 - val_loss: 0.4848 - val_accuracy: 0.8310 - val_precision: 0.8478 - val_recall: 0.8120 - val_precision_at_recall_2: 0.8364
Epoch 4/5
297/297 [==============================] - 38s 129ms/step - loss: 0.2623 - accuracy: 0.9122 - precision: 0.9

# Bidirectional LSTM Model


In [ ]:
BILSTMModel = Sequential()
BILSTMModel.add(Input(shape=x_train.shape[1]))
BILSTMModel.add(Embedding(len(tokenizer.word_index)+1,32))
BILSTMModel.add(Bidirectional(LSTM(100, return_sequences=True)))
BILSTMModel.add(GlobalMaxPooling1D())
BILSTMModel.add(Dense(4, activation='softmax'))
BILSTMModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall', tf.keras.metrics.PrecisionAtRecall(recall=0.8)])
    
BILSTMModel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 70, 32)            796384    
_________________________________________________________________
bidirectional (Bidirectional (None, 70, 200)           106400    
_________________________________________________________________
global_max_pooling1d (Global (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 804       
Total params: 903,588
Trainable params: 903,588
Non-trainable params: 0
_________________________________________________________________


In [ ]:
BILSTMModelResults = BILSTMModel.fit(x_train, y_train, epochs=5, batch_size=64,validation_split=0.1)

Epoch 1/5
297/297 [==============================] - 106s 318ms/step - loss: 1.1671 - accuracy: 0.5257 - precision: 0.5810 - recall: 0.2468 - precision_at_recall_3: 0.3057 - val_loss: 0.5215 - val_accuracy: 0.8220 - val_precision: 0.8494 - val_recall: 0.7850 - val_precision_at_recall_3: 0.4847
Epoch 2/5
297/297 [==============================] - 93s 313ms/step - loss: 0.4341 - accuracy: 0.8531 - precision: 0.8698 - recall: 0.8329 - precision_at_recall_3: 0.5605 - val_loss: 0.4620 - val_accuracy: 0.8338 - val_precision: 0.8548 - val_recall: 0.8139 - val_precision_at_recall_3: 0.6955
Epoch 3/5
297/297 [==============================] - 92s 311ms/step - loss: 0.3253 - accuracy: 0.8930 - precision: 0.9052 - recall: 0.8792 - precision_at_recall_3: 0.7256 - val_loss: 0.4948 - val_accuracy: 0.8286 - val_precision: 0.8408 - val_recall: 0.8125 - val_precision_at_recall_3: 0.7853
Epoch 4/5
297/297 [==============================] - 92s 311ms/step - loss: 0.2681 - accuracy: 0.9124 - precision: 0.

# Hyperparameter Tuning/ Fine Tuning

In [ ]:
BILSTMModel_2 = Sequential()
BILSTMModel_2.add(Input(shape=x_train.shape[1]))
BILSTMModel_2.add(Embedding(24887,300, weights=[embedding_matrix]))
BILSTMModel_2.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
BILSTMModel_2.add(GlobalMaxPooling1D())
BILSTMModel_2.add(Dense(50, activation='relu'))
BILSTMModel_2.add(Dropout(0.2))
BILSTMModel_2.add(Dense(4, activation='softmax'))
BILSTMModel_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall', tf.keras.metrics.PrecisionAtRecall(recall=0.8)])
    
BILSTMModel_2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 70, 300)           7466100   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 70, 100)           140400    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 204       
Total params: 7,611,754
Trainable params: 7,611,754
Non-trainable params: 0
____________________________________________

In [ ]:
BILSTMModel_2Results = BILSTMModel_2.fit(x_train, y_train, epochs=5, batch_size=64,validation_split=0.1)

Epoch 1/5
297/297 [==============================] - 189s 588ms/step - loss: 0.9542 - accuracy: 0.6120 - precision: 0.7025 - recall: 0.4175 - precision_at_recall_4: 0.3846 - val_loss: 0.4578 - val_accuracy: 0.8423 - val_precision: 0.8645 - val_recall: 0.8125 - val_precision_at_recall_4: 0.6685
Epoch 2/5
297/297 [==============================] - 175s 589ms/step - loss: 0.4126 - accuracy: 0.8594 - precision: 0.8746 - recall: 0.8427 - precision_at_recall_4: 0.7211 - val_loss: 0.4181 - val_accuracy: 0.8608 - val_precision: 0.8713 - val_recall: 0.8433 - val_precision_at_recall_4: 0.7959
Epoch 3/5
297/297 [==============================] - 175s 590ms/step - loss: 0.3319 - accuracy: 0.8904 - precision: 0.9023 - recall: 0.8774 - precision_at_recall_4: 0.8150 - val_loss: 0.4143 - val_accuracy: 0.8556 - val_precision: 0.8637 - val_recall: 0.8490 - val_precision_at_recall_4: 0.8517
Epoch 4/5
297/297 [==============================] - 175s 589ms/step - loss: 0.2640 - accuracy: 0.9140 - precision:

# Сохранение моделей

In [ ]:
SimpleRNNModel.save("/content/drive/MyDrive/4/Модели/SimpleRNNModel.h5")
SimpleLSTMModel.save("/content/drive/MyDrive/4/Модели/SimpleLSTMModel.h5")
SimpleGRUModel.save("/content/drive/MyDrive/4/Модели/SimpleGRUModel.h5")
BILSTMModel.save("/content/drive/MyDrive/4/Модели/BILSTMModel.h5")
BILSTMModel_2.save("/content/drive/MyDrive/4/Модели/BILSTMModel_2.h5")



In [ ]:
print(type(train['review'].values[1]))

<class 'str'>


In [ ]:
R = "I am sad"
R = tokenizer.texts_to_sequences(R)
R = sequence.pad_sequences(X, maxlen=max_len)

print('Shape of data tensor:', X.shape)

Shape of data tensor: (30160, 70)


In [ ]:
BILSTMModel_2.predict(R)

array([[3.5216109e-04, 6.8508966e-06, 9.9958366e-01, 5.7364370e-05],
       [1.9674664e-02, 1.1108759e-03, 8.4662527e-01, 1.3258918e-01],
       [8.8441791e-03, 3.1515816e-03, 9.8633331e-01, 1.6709854e-03],
       ...,
       [6.9408532e-04, 2.5791656e-03, 9.9155211e-01, 5.1746028e-03],
       [8.3590037e-04, 1.4830613e-01, 8.4371799e-01, 7.1399990e-03],
       [1.2121638e-01, 1.3621744e-02, 8.2696044e-01, 3.8201474e-02]],
      dtype=float32)

In [ ]:
print(np.argmax(BILSTMModel_2.predict(R), axis=-1))

[2 2 2 ... 2 2 2]


In [ ]:
label = train['label'].unique()
prediction = model.predict(R)
pred_name = label[np.argmax(prediction)]
print(pred_name)
#model.predict(R)



IndexError: ignored

In [ ]:
np.argmax(model.predict(R), axis=-1)

array([2, 0, 2, ..., 2, 2, 2])

In [ ]:
(model.predict(R) > 0.5).astype("int32")

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]], dtype=int32)